In [ ]:
!pip install reportlab

In [ ]:
import re
def extract_server_log_info(log_entry):
    # Define the primary regex pattern
    primary_pattern = re.compile(r'(?P<ip_address>\S+) - - \[(?P<timestamp>[^\]]+)\] "(?P<method>[A-Z]+) (?P<path>[^"]*) (?P<status>\d+) (?P<size>\d+) "(?P<referrer>[^"]*)" "(?P<user_agent>[^"]*)"')
    pattern = r'(?P<ip_address>[\d.]+) - - \[(?P<timestamp>[\w:/]+\s[+\-]\d{4})\] "(?P<method>[A-Z]+) (?P<path>[^"]*) HTTP/\d\.\d" (?P<status>\d+) (?P<size>\d+) "(?P<referrer>[^"]*)" "(?P<user_agent>[^"]*)"'

    # Define the alternative regex pattern
    alternative_pattern = re.compile(r'(?P<ip_address>\S+) - - \[(?P<timestamp>[^\]]+)\] "(?P<method>[A-Z]+) (?P<path>[^"]*)" (?P<status>\d+) (?P<size>-|\d+) "(?P<referrer>-|[^"]*)" "(?P<user_agent>[^"]*)"')
    alt2=re.compile(r'(?P<ip_address>[\d.]+) - - \[(?P<timestamp>[\w:/]+\s[+\-]\d{4})\] "(?P<method>[A-Z]+) (?P<path>.*?) HTTP/\d\.\d" (?P<status>\d+) (?P<size>\d+) "(?P<referrer>[^"]*)" "(?P<user_agent>[^"]*)"')
    alt3=re.compile(
    r'(?P<ip_address>[\d\.]+)\s-\s-\s\[(?P<timestamp>[^\]]+)\]\s"(?P<path>[^"]+)"\s(?P<status>\d+)\s(?P<size>\d+)\s"(?P<referrer>[^"]+)"\s"(?P<user_agent>[^"]+)"'
)
    alt4= re.compile(
    r'(?P<ip_address>[\d\.]+)\s-\s-\s\[(?P<timestamp>[^\]]+)\]\s"(?P<method>[A-Z]+) (?P<path>.*?) HTTP/1.1" (?P<status>\d+) (?P<size>\d+)\s"(?P<referrer>.*?)"\s"(?P<user_agent>[^"]*)"'
)
    alt5=re.compile(
    r'(?P<ip_address>[\d\.]+)\s-\s-\s\[(?P<timestamp>[^\]]+)\] "(?P<method>[A-Z]+) (?P<path>[^"]+) HTTP/1.1" (?P<status>\d+) (?P<size>\d+) "(?P<referrer>.*?)"\s"(?P<user_agent>[^"]*)"'
)
    # Try matching against the primary pattern
    match_primary = primary_pattern.match(log_entry)

    # If a match is found, create a dictionary with extracted information
    if match_primary:
        log_dict = match_primary.groupdict()
        log_dict['timestamp'] = pd.to_datetime(log_dict['timestamp'], format='%d/%b/%Y:%H:%M:%S %z', errors='coerce')
        return log_dict
    else:
        # Try matching against the alternative pattern
        match_alternative = alternative_pattern.match(log_entry)

        # If a match is found, create a dictionary with extracted information
        if match_alternative:
            log_dict = match_alternative.groupdict()
            log_dict['timestamp'] = pd.to_datetime(log_dict['timestamp'], format='%d/%b/%Y:%H:%M:%S %z', errors='coerce')
            return log_dict
        else:
            match2=alt2.match(log_entry)
            if match2:
                log_dict = match2.groupdict()
                log_dict['timestamp'] = pd.to_datetime(log_dict['timestamp'], format='%d/%b/%Y:%H:%M:%S %z', errors='coerce')
                return log_dict
            match3=alt3.match(log_entry)
            if match3:
                log_dict = match3.groupdict()
                log_dict['timestamp'] = pd.to_datetime(log_dict['timestamp'], format='%d/%b/%Y:%H:%M:%S %z', errors='coerce')
                return log_dict
            match4=alt4.match(log_entry)
            if match4:
                log_dict = match4.groupdict()
                log_dict['timestamp'] = pd.to_datetime(log_dict['timestamp'], format='%d/%b/%Y:%H:%M:%S %z', errors='coerce')
                return log_dict
            return None

def extract_firewall_log_info(line):
        log_parts = line.split(' ')
        # Extract relevant fields, considering variable number of fields
        log_dict = {part.split('=')[0]: part.split('=')[1] for part in log_parts if '=' in part}
        return log_dict

In [ ]:
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from datetime import datetime
import textwrap
def generate_report_attack_brief(firewall_entry, server_entry, chances):
    # Create a PDF file with a timestamped filename
    timestamp = server_entry['timestamp'].strftime("%Y_%m_%d_%H_%M_%S")
    pdf_filename = f"attack_report_{timestamp}.pdf"
    pdf = canvas.Canvas(pdf_filename, pagesize=letter)

    # Set up report title
    pdf.setFont("Helvetica-Bold", 24)  # Increased font size and bold
    pdf.setFillColorRGB(1, 0, 0)  # Red color
    pdf.drawCentredString(300, 750, "Attack Report")  # Center-aligned
    pdf.line(80, 745, 520, 745)  # Horizontal line after title

    # Add Firewall Entry Information
    pdf.setFont("Helvetica-Bold", 16)
    pdf.setFillColorRGB(1, 0, 0)  # Red color
    pdf.drawCentredString(300, 720, "Firewall Entry Information")
    pdf.line(80, 715, 520, 715)  # Horizontal line after firewall section

    # List firewall entry information using a loop
    y_position = 700
    for key, value in firewall_entry.items():
        # Check if there is enough space on the page, if not, start a new page
        if y_position < 70:
            pdf.showPage()
            y_position = 680  # Set starting y position on the new page

            # Set up report title for the new page
            pdf.setFont("Helvetica-Bold", 24)
            pdf.setFillColorRGB(1, 0, 0)
            pdf.drawCentredString(300, 750, "Attack Report")  # Center-aligned
            pdf.line(80, 745, 520, 745)  # Horizontal line after title

            # Add Firewall Entry Information header for the new page
            pdf.setFont("Helvetica-Bold", 16)
            pdf.setFillColorRGB(1, 0, 0)
            pdf.drawCentredString(300, 720, "Firewall Entry Information")
            pdf.line(80, 715, 520, 715)  # Horizontal line after firewall section

        # Wrap text to prevent overflow in x-axis
        pdf.setFont("Helvetica", 14)
        if len(f"{key}: {value}") > 55:
            wrap_text = textwrap.wrap(f"{key}: {value}", width=55)
            pdf.drawString(100, y_position, wrap_text[0])
            pdf.drawString(100, y_position-15, wrap_text[1])
            y_position -= 15
        else:
            pdf.drawString(100, y_position,f"{key}: {value}" )
        y_position -= 15

    # Add Server Entry Information
    pdf.setFont("Helvetica-Bold", 16)
    pdf.setFillColorRGB(1, 0, 0)  # Red color
    pdf.drawCentredString(300, y_position - 20, "Server Entry Information")
    pdf.line(80, y_position - 25, 520, y_position - 25)  # Horizontal line after server section

    # List server entry information using a loop
    y_position -= 40
    for key, value in server_entry.items():
        # Check if there is enough space on the page, if not, start a new page
        if y_position < 50:
            pdf.showPage()
            y_position = 680  # Set starting y position on the new page

            # Set up report title for the new page
            pdf.setFont("Helvetica-Bold", 24)
            pdf.setFillColorRGB(1, 0, 0)
            pdf.drawCentredString(300, 750, "Attack Report")  # Center-aligned
            pdf.line(80, 745, 520, 745)  # Horizontal line after title

            # Add Server Entry Information header for the new page
            pdf.setFont("Helvetica-Bold", 16)
            pdf.setFillColorRGB(1, 0, 0)
            pdf.drawCentredString(300, 720, "Server Entry Information")
            pdf.line(80, 715, 520, 715)  # Horizontal line after server section

        # Wrap text to prevent overflow in x-axis
        pdf.setFont("Helvetica", 14)
        if len(f"{key}: {value}") > 55:
            wrap_text = textwrap.wrap(f"{key}: {value}", width=55)
            pdf.drawString(100, y_position, wrap_text[0])
            pdf.drawString(100, y_position-15, wrap_text[1])
            y_position -= 15
        else:
            pdf.drawString(100, y_position,f"{key}: {value}" )
        y_position -= 15

    # Add Attack Probability with color encoding slabs
    color = get_color_encoding(chances)
    pdf.setFont("Helvetica-Bold", 18)
    pdf.setFillColorRGB(*color)
    pdf.drawCentredString(300, y_position - 20, f"Probability of Attack: {chances}%")
    pdf.line(80, y_position - 25, 520, y_position - 25)  # Horizontal line after attack probability

    # Save the PDF
    pdf.save()

def get_color_encoding(chances):
    # Color encoding based on provided intervals
    if  chances <= 60:
        return 0.2, 0.8, 0.2  # Green
    elif 60 < chances <= 70:
        return 1, 0.5, 0  # Orange
    else:
        return 1, 0, 0  # Red


def generate_anomaly_report(anomaly_chances, firewall_df):
    # Count the number of anomalies detected
    num_anomalies = sum(1 for chance in anomaly_chances if chance < 0)

    # Create a PDF file with a timestamped filename
    timestamp = firewall_df['timestamp'].strftime("%Y_%m_%d_%H_%M_%S")
    pdf_filename = f"anomaly_report_{timestamp}.pdf"
    pdf = canvas.Canvas(pdf_filename, pagesize=letter)

    # Set up report title
    pdf.setFont("Helvetica-Bold", 24)
    pdf.setFillColorRGB(1, 0, 0)
    pdf.drawCentredString(300, 750, "Anomaly Report")
    pdf.line(80, 745, 520, 745)

    # Add summarized information on top
    pdf.setFont("Helvetica", 14)
    pdf.setFillColorRGB(0, 0, 0)  # Black color
    pdf.drawString(100, 720, f"Number of Anomalies Detected: {num_anomalies}")

    # Sort anomaly_chances and firewall_df based on anomaly_chances in descending order
    sorted_data = sorted(zip(anomaly_chances, firewall_df.to_dict(orient='records')),
                         key=lambda x: x[0], reverse=True)

    y_position = 700

    # Add detailed information in a table
    pdf.setFont("Helvetica-Bold", 16)
    pdf.setFillColorRGB(1, 0, 0)
    pdf.drawCentredString(300, y_position - 20, "Anomaly Details")
    pdf.line(80, y_position - 25, 520, y_position - 25)

    y_position -= 40

    for chance, entry in sorted_data:
        # Check if there is enough space on the page, if not, start a new page
        if y_position < 50:
            pdf.showPage()
            y_position = 680  # Set starting y position on the new page
            pdf.setFont("Helvetica-Bold", 24)
            pdf.setFillColorRGB(1, 0, 0)
            pdf.drawCentredString(300, 750, "Anomaly Report")
            pdf.line(80, 745, 520, 745)

            pdf.setFont("Helvetica-Bold", 16)
            pdf.setFillColorRGB(1, 0, 0)
            pdf.drawCentredString(300, 720, "Anomaly Details")
            pdf.line(80, 715, 520, 715)

        # Wrap text to prevent overflow in x-axis
        pdf.setFont("Helvetica", 14)
        pdf.setFillColorRGB(*(1, 0, 0))
        text = f"Firewall Entry: {entry['firewall_entry']}, Server Entry: {entry['server_entry']}, Probability: {chance}%"
        if len(text) > 55:
            wrap_text = textwrap.wrap(text, width=55)
            pdf.drawString(100, y_position, wrap_text[0])
            pdf.drawString(100, y_position - 15, wrap_text[1])
            y_position -= 15
        else:
            pdf.drawString(100, y_position, text)
        y_position -= 15

    # Save the PDF
    pdf.save()



In [ ]:
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from datetime import datetime
import textwrap

def generate_packet_alert(packet, prediction):
    # Create a PDF file with a timestamped filename
    timestamp = datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
    pdf_filename = f"packet_alert_{timestamp}.pdf"
    pdf = canvas.Canvas(pdf_filename, pagesize=letter)

    # Set up report title
    pdf.setFont("Helvetica-Bold", 24)
    pdf.setFillColorRGB(1, 0, 0)  # Red color
    pdf.drawCentredString(300, 750, "Packet Alert")  # Center-aligned
    pdf.line(80, 745, 520, 745)  # Horizontal line after title

    # Add Packet Information
    pdf.setFont("Helvetica-Bold", 16)
    pdf.setFillColorRGB(1, 0, 0)  # Red color
    pdf.drawCentredString(300, 720, "Packet Information")
    pdf.line(80, 715, 520, 715)  # Horizontal line after packet section

    # List packet information using a loop
    y_position = 700
    for key, value in packet.items():
        # Check if there is enough space on the page, if not, start a new page
        if y_position < 70:
            pdf.showPage()
            y_position = 680  # Set starting y position on the new page

            # Set up report title for the new page
            pdf.setFont("Helvetica-Bold", 24)
            pdf.setFillColorRGB(1, 0, 0)
            pdf.drawCentredString(300, 750, "Packet Alert")  # Center-aligned
            pdf.line(80, 745, 520, 745)  # Horizontal line after title

            # Add Packet Information header for the new page
            pdf.setFont("Helvetica-Bold", 16)
            pdf.setFillColorRGB(1, 0, 0)
            pdf.drawCentredString(300, 720, "Packet Information")
            pdf.line(80, 715, 520, 715)  # Horizontal line after packet section

        # Wrap text to prevent overflow in x-axis
        pdf.setFont("Helvetica", 14)
        if len(f"{key}: {value}") > 55:
            wrap_text = textwrap.wrap(f"{key}: {value}", width=55)
            pdf.drawString(100, y_position, wrap_text[0])
            pdf.drawString(100, y_position - 15, wrap_text[1])
            y_position -= 15
        else:
            pdf.drawString(100, y_position, f"{key}: {value}")
        y_position -= 15

    # Add Prediction Information
    pdf.setFont("Helvetica-Bold", 16)
    pdf.setFillColorRGB(1, 0, 0)  # Red color
    pdf.drawCentredString(300, y_position - 20, "Prediction Information")
    pdf.line(80, y_position - 25, 520, y_position - 25)  # Horizontal line after prediction section

    # Display the prediction
    pdf.setFont("Helvetica", 14)
    pdf.drawString(100, y_position - 40, f"Prediction: {prediction}")

    # Save the PDF
    pdf.save()



In [ ]:
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer
from reportlab.lib import colors
from reportlab.lib.styles import getSampleStyleSheet
from datetime import datetime

def generate_report_attack(firewall_entry, server_entry, chances):
    # Create a Story to hold the content
    story = []

    # Set up report title
    style = getSampleStyleSheet()["Title"]
    story.append(Paragraph("Attack Report", style))
    story.append(Spacer(1, 12))  # Add some space after the title
    story.append(Spacer(1, 12))  # Add some space after the section
    story.append(Paragraph("-"*76, style))  # Horizontal line
    story.append(Spacer(1, 12))  # Add some space before the line
    # Add Firewall Entry Information
    style = getSampleStyleSheet()["Normal"]
    story.append(Paragraph("Firewall Entry Information", style))
    style = getSampleStyleSheet()["BodyText"]
    for key, value in firewall_entry.items():
        
        story.append(Paragraph(f"{key}: {value}", style))

    # Add horizontal line after firewall section
    story.append(Spacer(1, 12))  # Add some space after the section
    story.append(Paragraph("-"*76, style))  # Horizontal line
    story.append(Spacer(1, 12))  # Add some space before the line

    # Add Server Entry Information
    style = getSampleStyleSheet()["Normal"]
    story.append(Paragraph("Server Entry Information", style))
    style = getSampleStyleSheet()["BodyText"]
    for key, value in server_entry.items():
        story.append(Paragraph(f"{key}: {value}", style))

    # Add horizontal line after server section
    story.append(Spacer(1, 12))  # Add some space after the section
    story.append(Paragraph("-"*76, style))  # Horizontal line
    story.append(Spacer(1, 12))  # Add some space before the line

    # Add Attack Probability with color encoding slabs
    color = get_color_encoding(chances)
    style = getSampleStyleSheet()["Heading1"]
    style.textColor = colors.red  # Change heading color to red
    story.append(Paragraph(f"Probability of Attack: {chances}%", style))

    # Add horizontal line after attack probability
    story.append(Spacer(1, 12))  # Add some space after the section
    story.append(Paragraph("-"*76, style))  # Horizontal line
    story.append(Spacer(1, 12))  # Add some space before the line

    return story

def generate_Master_report(attack_firewalls, attack_servers, attack_chances, firewalls, servers, blocked_requests):
    # Create a PDF file with a timestamped filename
    timestamp = datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
    pdf_filename = f"Master{timestamp}.pdf"

    # Create a Story to hold the content
    story = []

    # Set up report title
    style = getSampleStyleSheet()["Title"]
    story.append(Paragraph("Master Report", style))
    story.append(Spacer(1, 12))  # Add some space after the title
    story.append(Spacer(1, 12))  # Add some space after the section
    story.append(Paragraph("-"*76, style))  # Horizontal line
    story.append(Spacer(1, 12))  # Add some space before the line
    # Add summary information
    style = getSampleStyleSheet()["Heading1"]
    style.textColor = colors.red  # Change heading color to red
    story.append(Paragraph("Summary Information", style))
    story.append(Paragraph(f"Number of Attacks: {len(attack_firewalls)}", style))
    story.append(Paragraph(f"Number of Requests: {len(servers)}", style))
    
    # Add horizontal line after summary information
    story.append(Spacer(1, 12))  # Add some space after the section
    story.append(Paragraph("-"*76, style))  # Horizontal line
    story.append(Spacer(1, 12))  # Add some space before the line

    # Add Blocked Firewalls Information
    style = getSampleStyleSheet()["Heading1"]
    style.textColor = colors.red  # Change heading color to red
    story.append(Paragraph("Blocked Firewalls", style))
    for firewall in blocked_requests:
        story.append(Paragraph(f"Blocked Firewall: {firewall}", style))

    # Add horizontal line after blocked firewalls
    story.append(Spacer(1, 12))  # Add some space after the section
    story.append(Paragraph("-"*76, style))  # Horizontal line
    story.append(Spacer(1, 12))  # Add some space before the line

    # Add Attack Information
    style = getSampleStyleSheet()["Heading1"]
    style.textColor = colors.red  # Change heading color to red
    story.append(Paragraph("Attack Information", style))
    story.append(Spacer(1, 12))  # Add some space after the section
    story.append(Paragraph("-"*76, style))  # Horizontal line
    story.append(Spacer(1, 12))  # Add some space before the line

    # Generate separate sections for each attack and add them to the master report
    for (firewall_entry, server_entry, chances) in zip(attack_firewalls, attack_servers, attack_chances):
        attack_section = generate_report_attack(firewall_entry, server_entry, chances)
        story.extend(attack_section)
        story.append(Spacer(1, 12))  # Add some space between sections

    # Save the PDF
    doc = SimpleDocTemplate(pdf_filename, pagesize=letter)
    doc.build(story)

    print(f"Master Report generated successfully: {pdf_filename}")

In [ ]:
from collections import Counter
def generate_summary_report(attack_firewalls, servers, blocked_requests,p,ana):
    # Create a PDF file with a timestamped filename
    timestamp = datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
    pdf_filename = f"Summary{timestamp}.pdf"

    # Create a Story to hold the content
    story = []
    num_anomalies = sum(1 for chance in ana if chance < 0)

    # Set up report title
    style = getSampleStyleSheet()["Title"]
    story.append(Paragraph(" Summarized Report ", style))
    story.append(Spacer(1, 12))  # Add some space after the title
    story.append(Spacer(1, 12))  # Add some space after the section
    story.append(Paragraph("-"*76, style))  # Horizontal line
    story.append(Spacer(1, 12))  # Add some space before the line
    # Add summary information
    style = getSampleStyleSheet()["Heading1"]
    style.textColor = colors.red  # Change heading color to red
    story.append(Paragraph("Summary Information", style))
    story.append(Paragraph(f"Number of Attacks: {len(attack_firewalls)}", style))
    story.append(Paragraph(f"Number of Requests: {len(servers)}", style))
    story.append(Paragraph(f"Blocked Requests: {len(blocked_requests)}", style))
    story.append(Paragraph(f"Anamolies Detected: {num_anomalies}", style))
    
    unique_values_counter = Counter(p)
    total_values = len(p)

    # Add the Percentage information to the report
    style = getSampleStyleSheet()["Heading1"]
    style.textColor = colors.blue  # Change heading color to blue
    story.append(Paragraph("Unique Values Percentage", style))

    for value, count in unique_values_counter.items():
        percentage = (count / total_values) * 100
        story.append(Paragraph(f"{value}: {percentage:.2f}%", style))

    # Add horizontal line after summary information
    story.append(Spacer(1, 12))  # Add some space after the section
    story.append(Paragraph("-"*76, style))  # Horizontal line
    story.append(Spacer(1, 12))  # Add some space before the line

    doc = SimpleDocTemplate(pdf_filename, pagesize=letter)
    doc.build(story)

    print(f"Summarised Report generated successfully: {pdf_filename}")

In [ ]:
import tensorflow as tf
import tensorflow_text as text
import urllib
bert_model=tf.saved_model.load('/kaggle/input/cyber-detector/CyberCrime_bert')
blacklist=[]
from tqdm import tqdm

def preprocess(d):
        d = str(urllib.parse.unquote(d))
        d=d.replace('\n','')
        d=d.replace('?','? ')
        d=d.replace(')',') ')
        d=d.replace('(','( ')
        d=d.replace('{','{ ')
        d=d.replace('}','} ')
        return d
nan=None
sample_df={'logver': '506141727',
 'tz': '"UTC+5:30"',
 'devname': '"FGT3600C_HA"',
 'devid': '"FG3K6C3A15800081"',
 'vd': '"root"',
 'date': '2023-10-30',
 'time': '23:54:54',
 'logid': '"0000000013"',
 'type': '"traffic"',
 'subtype': '"forward"',
 'level': '"notice"',
 'eventtime': '1698690294',
 'srcip': '103.81.182.133',
 'srcport': '30390',
 'srcintf': '"LLB-',
 'srcintfrole': '"wan"',
 'dstip': '172.26.2.51',
 'dstport': '443',
 'dstintf': '"Local_LAN"',
 'dstintfrole': '"undefined"',
 'poluuid': '"3367bf4c-74ff-51e8-3e96-72b0684b3e81"',
 'sessionid': '1943252826',
 'proto': '6',
 'action': '"client-rst"',
 'policyid': '49',
 'policytype': '"policy"',
 'service': '"HTTPS"',
 'dstcountry': '"Reserved"',
 'srccountry': '"India"',
 'trandisp': '"noop"',
 'duration': '13',
 'sentbyte': '1097',
 'rcvdbyte': '9182',
 'sentpkt': '11',
 'appcat': '"unscanned"\n',
 'rcvdpkt': nan,
 'wanin': nan,
 'wanout': nan,
 'lanin': nan,
 'lanout': nan,
 'sentdelta': nan,
 'rcvddelta': nan,
 'crscore': nan,
 'craction': nan,
 'crlevel': nan,
 'logdesc': nan,
 'hostname': nan,
 'msg': nan,
 'app': nan,
 'cpu': nan,
 'mem': nan,
 'totalsession': nan,
 'disk': nan,
 'bandwidth': nan,
 'setuprate': nan,
 'disklograte': nan,
 'fazlograte': nan,
 'user': nan,
 'ui': nan,
 'status': nan,
 'count': nan,
 'reason': nan,
 'log': nan,
 'eventtype': nan,
 'severity': nan,
 'attack': nan,
 'direction': nan,
 'attackid': nan,
 'profile': nan,
 'ref': nan,
 'incidentserialno': nan,
 'utmaction': nan,
 'countips': nan}
def anveshan(firewalls,servers):
    attack_f=[]
    attack_s=[]
    block_s=[]
    
    c=[]
    for log1 in tqdm(servers):
     if blacklist!=[] and log1['ip_address'] in blacklist:
            block_s.append(log1)
     else:
        url=log1['path']
        p=bert_model(tf.constant([preprocess(url)]))
        chance=p[0][1].numpy()*100
        if chance>50:
            try:
                log2=firewalls.loc[log1['timestamp']].iloc[0].to_dict()
            except:
                log2=sample_df
            generate_report_attack_brief(log2,log1,chance)
            pratikriya(log2,log1)
            attack_f.append(log2)
            attack_s.append(log1)
            c.append(chance)
    return attack_f,attack_s,c,block_s
            

In [ ]:
import joblib
import pandas as pd
pack=joblib.load('/kaggle/input/packet-analyser/random_many_attack_forest_model.joblib')
firewall_analyser=joblib.load('/kaggle/input/packet-analyser/firewall_model_analyser.joblib')
firewall_Anamoly=joblib.load('/kaggle/input/packet-analyser/anamoly.joblib')
def pahchan(packet_url):
    columns = ["duration", "protocol_type", "service", "flag", "src_bytes", "dst_bytes", "land", "wrong_fragment", "urgent", "hot", "num_failed_logins", "logged_in", "num_compromised", "root_shell", "su_attempted", "num_root", "num_file_creations", "num_shells", "num_access_files", "num_outbound_cmds", "is_host_login", "is_guest_login", "count", "srv_count", "serror_rate", "srv_serror_rate", "rerror_rate", "srv_rerror_rate", "same_srv_rate", "diff_srv_rate", "srv_diff_host_rate", "dst_host_count", "dst_host_srv_count", "dst_host_same_srv_rate", "dst_host_diff_srv_rate", "dst_host_same_src_port_rate", "dst_host_srv_diff_host_rate", "dst_host_serror_rate", "dst_host_srv_serror_rate", "dst_host_rerror_rate", "dst_host_srv_rerror_rate"]
    df=pd.read_csv(packet_url,header=None, names=columns)
    d2=df.drop(['protocol_type', 'service', 'flag'], axis=1)
    print("IDS processing done")
    #Preprocess
    chan=pack.predict(d2)
    #Dectector
    #print(chan)
    for i in range(len(chan)):
        if chan[i]!='normal.':
            generate_packet_alert(df.iloc[i].to_dict(),chan[i])
    return chan
def anamoly(f):
    selected_cols=['srcip', 'srcport', 'srcintf', 'dstip', 'dstport', 'dstintf',
                    'proto', 'action', 'service', 'duration', 'sentbyte',
                    'rcvdbyte', 'sentpkt', 'appcat', 'rcvdpkt', 'wanin', 'wanout',
                    'lanin', 'lanout', 'sentdelta', 'rcvddelta', 'countips']
    p=f[selected_cols]
    results=firewall_Anamoly.predict(p)
    return results

def pratikriya(f,s):
    blacklist.append(s['ip_address'])
    punar_sthapana(f,s)
def punar_sthapana(f,s):
    #Run Test case
    #Validate the files and firewall
    pass

def Kavach(f,s,ids):
    firewall=[]
    server=[]
    with open(f) as f2:
      for i in f2:
        fl=extract_firewall_log_info(i)
        firewall.append(fl)
    print("Firewall Processing done")
    with open(s) as s2:
      for j in s2:
        sl=extract_server_log_info(j)
        server.append(sl)
    print("Server Processing done")
            
    firewall_df=pd.DataFrame(firewall)
    firewall_df['timestamp'] = pd.to_datetime(firewall_df['date'] + ' ' + firewall_df['time'], format='%Y-%m-%d %H:%M:%S', utc=True)
    p=pahchan(ids)
    ana=anamoly(firewall_df)
    generate_anomaly_report(ana, firewall_df)
    attack_f,attack_s,attack_c,block_s=anveshan(firewall_df,server)
    print("Reports ready")
    generate_Master_report(attack_f, attack_s, attack_c, firewall, server, block_s)
    print("Detailed Master Report Ready")
    generate_summary_report(attack_firewalls, servers, blocked_requests, p,ana)
    print("Summarised Report Ready")

In [ ]:
Kavach('/kaggle/input/cyber-security/31Oct2023-messages.txt','/kaggle/input/cyber-security/ssl_access_log-20231107','/kaggle/input/threats/Group-5/kddcup.testdata.unlabeled_10_percent')